# Analyse PDF file

..

Cette première section a pour objectif de charger les modules Python nécessaire pour la bonne exécution du notebook. \
Dans la dernière partie, il sera nécessaire de saisir la valeur de la variable **pdfFile** correspondant au chemin du fichier PDF à analyser. 

In [113]:
from colorama import init, Fore, Back, Style
import os
import json
import pandas as pd
import hashlib
import peepdf

import re
import subprocess
from IPython.display import display, HTML

#from pandas.io.json import json_normalize
#import ipaddress as ip
#from msticpy.transform.iocextract import IoCExtract
#import msticpy as mp
#mp.init_notebook(globals(), verbosity=0)
#ti = mp.TILookup()
#ioc_extract = IoCExtract()

#Expand the width of the cells
display(HTML("<style>.container { width:90% !important; }</style>"))

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

def convertListToDict(data):
    it = iter(data)
    return dict(zip(it, it))

pdfFile = "/home/secubian/Desktop/Cases/pdfs/61f0e80b2a74bf26d2089e09e779b0514b4d7e324de6f968bb224cbe6f2fab91.pdf"
pdfFile = "/home/secubian/Desktop/Cases/pdfs/1796aef0940e800bcb2556782f92a7874422bbdfdda24e6658e43db4b0916850.pdf"
pdfFile = "/home/secubian/Desktop/Cases/pdfs/0062742b752ac42b2812ee2670b18943322856268c8d3e23e7f97656bd41969e.pdf"
pdfFile = "/home/secubian/Desktop/Cases/pdfs/907e75030b0e09cec6524f612f1c7439b5260b57b43d515968f81ba69278ba77.pdf"
pdfFile = "/home/secubian/Desktop/Cases/pdfs/d1c2cc0ca653df8ddb46c1337a5972eaceb81ea924e8ebdb7af0699a7ab909fd.pdf"
pdfFile = "/home/secubian/Desktop/Cases/pdfs/embedded_xls_180888bfe38fb7843583c469b893806ff0f9bb169c5e7be900779764742be229.pdf"



# https://raw.githubusercontent.com/intelowlproject/IntelOwl/master/api_app/analyzers_manager/file_analyzers/pdf_info.py
# https://github.com/hiddenillusion/AnalyzePDF/blob/master/AnalyzePDF.py

pdfFileReport = {}
pdfFileReport['version'] = []

pdf_parser = peepdf.PDFCore.PDFParser()
ret, pdf = pdf_parser.parse(pdfFile, True)

pdfFileReport['file'] = pdf.getStats().get('File',[])
pdfFileReport['md5'] = pdf.getStats().get('MD5',[])
pdfFileReport['sha256'] = pdf.getStats().get('SHA256',[])
pdfFileReport['detection'] = pdf.getStats().get('Detection',[])
pdfFileReport['pdf_version'] = pdf.getStats().get('Version',[])
if pdf.getStats().get('Binary',[]): pdfFileReport['binary'] = pdf.getStats().get('Binary',[])
if pdf.getStats().get('Encrypted',[]): pdfFileReport['encrypted'] = pdf.getStats().get('Encrypted',[])
if pdf.getStats().get('Encryption Algorithms',[]): pdfFileReport['encryption_algorithm'] = pdf.getStats().get('Encryption Algorithms',[])
if pdf.getStats().get('Comments',[]): pdfFileReport['comments'] = pdf.getStats().get('Comments',[])

pdfFileReport['pdfFile_version_count'] = len(pdf.getStats().get("Versions",[]))

for version in pdf.getStats().get("Versions",[]):
    pdfVersion = {}
    if version.get("Actions", []):  pdfVersion['actions'] = version.get("Actions", [])
    if version.get("Compressed Objects", []): pdfVersion['compressed_objects'] = version.get("Compressed Objects", [])
    if version.get("Encoded", []): pdfVersion['encoded_objects'] = version.get("Encoded", [])
    if version.get("Elements", []): pdfVersion['elements'] = version.get("Elements", [])
    if version.get("Events", []): pdfVersion['events'] = version.get("Events", [])
    if version.get("Objects with JS code", []): pdfVersion['objects_with_js'] = version.get("Objects with JS code", [])
    if version.get("Streams", []): pdfVersion['streams'] = version.get("Streams", [])
    if pdf.getURIs(): pdfVersion['URI'] = pdf.getURIs()
    if pdf.getURLs(): pdfVersion['URL'] = pdf.getURLs()
    if version.get("Vulns", []): pdfVersion['vulns'] = version.get("Vulns", [])

    pdfFileReport['version'].append(pdfVersion)

pdf.getStats()

{'File': 'embedded_xls_180888bfe38fb7843583c469b893806ff0f9bb169c5e7be900779764742be229.pdf',
 'MD5': '60fcb97dd9e3c0ce787db9885350d049',
 'SHA1': 'a115181ea8b31caf5187a3837847e07c334219f5',
 'SHA256': '180888bfe38fb7843583c469b893806ff0f9bb169c5e7be900779764742be229',
 'Size': '200742',
 'Detection': [],
 'Detection report': '',
 'Version': '1.6',
 'Binary': 'True',
 'Linearized': 'False',
 'Encrypted': 'False',
 'Encryption Algorithms': [],
 'Updates': '0',
 'Objects': '10',
 'Streams': '8',
 'URIs': '0',
 'Comments': '0',
 'Errors': [],
 'Versions': [{'Catalog': '2',
   'Info': '3',
   'Objects': ['10', [1, 2, 12, 13, 14, 21, 22, 24, 25, 26]],
   'Compressed Objects': None,
   'Errors': ['8', [12, 13, 14, 21, 22, 24, 1, 26]],
   'Streams': ['8', [12, 13, 14, 21, 22, 24, 26, 1]],
   'Xref Streams': ['1', [26]],
   'Object Streams': ['1', [1]],
   'Encoded': ['8', [12, 13, 14, 21, 22, 24, 26, 1]],
   'Decoding Errors': ['7', [12, 13, 14, 22, 24, 26, 1]],
   'URIs': None,
   'Objects w

## Méta Données

La toute première étape consiste à obtenir les méta-données du fichier PDF. \
Ces dernières vont nous permettre d'interroger les bonnes de Threat Intel, afin d'identifier rapidement tout ficher malveillant.

In [98]:
if (pdfFileReport):
    print(Fore.GREEN + "[!] PDF file analyse success")
    print(f"File Name : {pdfFileReport['file']}")
    print(f"File Hash [md5] : {pdfFileReport['md5']}")
    print(f"File Hash [sha256] : {pdfFileReport['sha256']}")
else:
    print(Fore.RED + "[✓] PDF file analyse failed.")


[!] PDF file analyse success
File Name : embedded_xls_180888bfe38fb7843583c469b893806ff0f9bb169c5e7be900779764742be229.pdf
File Hash [md5] : 60fcb97dd9e3c0ce787db9885350d049
File Hash [sha256] : 180888bfe38fb7843583c469b893806ff0f9bb169c5e7be900779764742be229


[TODO] recherche en Threat Intel des hashs identifiés.

## Analyse des sections du document PDF 



Les sections les plus interessantes sont les suivantes : 
- **EmbeddedFile** : Cette section énumère les fichiers présents au sein du fichier PDF. Il peut s'agir par exemple de script ou d'image.
- **Javascript** ou **JS** : Il s'agit d'une référence à du Javascript pouvant être exécuté à l'ouverture du fichier PDF.
- **Names** : Noms des éventuels fichiers auxquels le fichier PDF pourrait faire référence.
- **OpenAction** ou **Launch** : Il s'agit d'une fonction exécutée à l'ouverture du fichier pdf. Elle peut être utilisée pour exécuter un script par exemple.
- **URI** ou **SubmitForm** : Cette section recense les éventuelles références à des URL pouvant conduire le lecteur du fichier PDF à télécharger des scripts/binaires malveillants.



Maintenant que nous avons présenté les sections potentiellement suspectes. \
Ces dernières vont être extraites via l'outil **peepdf**, afin de compléter l'analyse.

### Identification de code Javascript

In [99]:
artefact_js = []
for version in pdf.getStats().get("Versions",[]):
    if version.get("Objects with JS code", []): 
        artefact_js.append(version.get("Objects with JS code", []))

if artefact_js:
    print(Fore.RED + "[✓] Some Javascript code detected")
    print(artefact_js)
else:
    print(Fore.GREEN + "[!] No Javascript code detected")

[!] No Javascript code detected


### Identification des sections OpenAction/Launch

In [100]:
artefact_openaction = []
for version in pdf.getStats().get("Versions",[]):
    if version.get("Actions", []):  
        artefact_openaction.append(version.get("Actions", []))
    if version.get("Events", []):  
        artefact_openaction.append(version.get("Events", []))

if artefact_openaction:
    print(Fore.RED + "[✓] Some OpenAction/Launch detected")
    print(artefact_openaction)
else:
    print(Fore.GREEN + "[!] No OpenAction/Launch detected")


[✓] Some OpenAction/Launch detected
[{'/OpenAction': [2], '/Names': [2], '/AcroForm': [2]}]


### Identification des URI/URL

In [101]:
artefact_uri = []
for version in pdf.getStats().get("Versions",[]):
    if pdf.getURIs(): 
        for uri in pdf.getURIs():
            if uri: artefact_uri.append(uri)
    if pdf.getURLs(): 
        for url in pdf.getURLs():
            if url: artefact_uri.append(url)

if (artefact_uri):
    print(Fore.RED + "[✓] Some URI/URL detected")
    print(artefact_uri)
else:
    print(Fore.GREEN + "[!] No URI/URL detected")


[!] No URI/URL detected


### Identification des éventuelles vulnérabilités

In [102]:
artefact_vulns = []
for version in pdf.getStats().get("Versions",[]):
    if version.get("Vulns", []):  
        artefact_openaction.append(version.get("Vulns", []))


if (artefact_vulns):
    print(Fore.RED + "[✓] Some vulnerabilities detected")
    print(artefact_vulns)
else:
    print(Fore.GREEN + "[!] No vulnerabilities detected")


[!] No vulnerabilities detected


### Identification de fichier(s) présent(s) dans le PDF

Cette analyse permet de mettre en avant de potentiel(s) fichier(s) malveillant(s) caché(s) au sein du fichier pdf.

In [114]:
artefact_embedded = []
for version in pdf.getStats().get("Versions",[]):
    if version.get("Elements", []):  
        artefact_embedded.append(version.get("Elements", []))


if (artefact_embedded):
    print(Fore.RED + "[✓] Some embedded files detected")
    print(artefact_embedded)
else:
    print(Fore.GREEN + "[!] No embedded files detected")

[✓] Some embedded files detected
[{'/EmbeddedFile': [22]}]


Il est tout à fait possible d'extraire les documents incrustés via la commande **pdf-parser**.

In [118]:
objectID = 22
os.system(f"pdf-parser.py --object {objectID} -f -w -d {pdfFile}_{objectID} {pdfFile}")

obj 22 0
 Type: /EmbeddedFile
 Referencing: 
 Contains stream

  <<
    /Filter /FlateDecode
    /Type /EmbeddedFile
    /Length 184388
  >>




0

Le type de fichier extrait, peut désormais être identifié via la commande **file**.

In [119]:
os.system(f"file {pdfFile}_{objectID}")

/home/secubian/Desktop/Cases/pdfs/embedded_xls_180888bfe38fb7843583c469b893806ff0f9bb169c5e7be900779764742be229.pdf_22: CDFV2 Encrypted


0

### Analyse avancée des objets

Dans un second temps, nous allons identifier si le contenu est chiffré, encodé ou toutes autres sections suspectes.

In [104]:
artefact_encoded = []
artefact_streams = []

for version in pdf.getStats().get("Versions",[]):
    if version.get("Compressed Objects", []): pdfVersion['compressed_objects'] = version.get("Compressed Objects", [])
    if version.get("Encoded", []): artefact_encoded.append(version.get("Encoded", []))
    if version.get("Streams", []): artefact_streams.append(version.get("Streams", []))

if artefact_encoded:
    print(Fore.RED + "[✓] Some encoded section detected.")
    print(artefact_encoded)

if artefact_streams:
    print(Fore.RED + "[✓] Some streams detected. Result contains stream ID.")
    print(artefact_streams)


[✓] Some encoded section detected.
[['8', [12, 13, 14, 21, 22, 24, 26, 1]]]
[✓] Some streams detected. Result contains stream ID.
[['8', [12, 13, 14, 21, 22, 24, 26, 1]]]


Certains objets ont été identifiés comme encodé ou chiffré. \
Il peut être interessant de réaliser une analyse approfondie en visualisant le contenu de ces objets. \
Pour cela, vous devrez saisir la valeur **objectID**.

In [111]:
# Advanced Analyse about objects
# Encoded object : 1, 2, 4
## source : https://github.com/jesparza/peepdf/blob/master/peepdf.py
objectID = 22
object = pdf.getObject(objectID)
objectReport = {}


objectReport['type'] = object.elements['/Type'].getValue()
objectReport['length'] = object.elements['/Length'].getValue()
objectReport['filter'] = object.elements['/Filter'].getValue()
print(objectReport)
#object.getRawValue()



{'type': '/EmbeddedFile', 'length': '184388', 'filter': '/FlateDecode'}
